**Summary of the Experiment:**

Dataset Name : CommonsenseQA

There were two files given namely train_rand_split.jsonl and dev_rand_split.jsonl

train_rand_split is used as training set and dev_rand_split as Validation set

Data contains following columns:
1. Question id
2. Question
3. Answer key

Following data pre-processing steps are carried out:

1. Question column contains the context followed by question itself was captured in the form of dictionary : Here context and question has broken into two separate columns as question includes context as well

2. The Answer key contains either of one of the options from [A,B,C,D,E] as it's just a label captured in the form of character and do not reveal much infomation about the answer itself. So capture the actual answer text label key is mapped to appropriate answer text 

Source : Question
Target : Answer text

Changes made to the code : Added glove emmbedding while building a vocab

Final observations : Compared to Model1 (Seq2Seq) Model2 (Seq2Seq with attention) performs well on tarining set. However model still overfits and need addtional architectural changes and tunning needed to imporove the performance

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/END-NLP/Assignment9-encoder-decoder

/content/drive/My Drive/END-NLP/Assignment9-encoder-decoder


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd 
import torch.nn.functional as F
from torchtext.data import Example,Field, BucketIterator,Dataset

import spacy
import numpy as np

import random
import math
import time

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [ ]:
train=pd.read_json('train_rand_split.jsonl',lines=True)
test=pd.read_json('dev_rand_split.jsonl',lines=True)

In [ ]:
def fun_map_answerkey(dt):
    l=list(dt['question'].values())[1]
    for i in l:
      if(dt['answerKey'] in i.values()):
        return(str(list(i.values())[1]))

In [ ]:
train['ans_text']=train[['question','answerKey']].apply(fun_map_answerkey,axis=1)

In [ ]:
def fun_unlist(a):
    l=list(a.values())
    l1=[ x['label'] +"." + x['text'] for x in l[1:][0]]
    return(pd.Series([ str(l[2]+" "+" ".join(l1)), str(l[0])]))

In [ ]:
train[['question_extracted','question_concept']]=train['question'].apply(lambda x : fun_unlist(x))

In [ ]:
train.head()

,answerKey,id,question,ans_text,question_extracted,question_concept
0,A,075e483d21c29a511267ef62bedc0461,"{'question_concept': 'punishing', 'choices': [{'label': 'A', 'text': 'ignore'}, {'label': 'B', 'text': 'enforce'}, {'label': 'C', 'text': 'authoritarian'}, {'label': 'D', 'text': 'yell at'}, {'label': 'E', 'text': 'avoid'}], 'stem': 'The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change?'}",ignore,"The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change? A.ignore B.enforce C.authoritarian D.yell at E.avoid",punishing
1,B,61fe6e879ff18686d7552425a36344c8,"{'question_concept': 'people', 'choices': [{'label': 'A', 'text': 'race track'}, {'label': 'B', 'text': 'populated areas'}, {'label': 'C', 'text': 'the desert'}, {'label': 'D', 'text': 'apartment'}, {'label': 'E', 'text': 'roadblock'}], 'stem': 'Sammy wanted to go to where the people were. Where might he go?'}",populated areas,Sammy wanted to go to where the people were. Where might he go? A.race track B.populated areas C.the desert D.apartment E.roadblock,people
2,A,4c1cb0e95b99f72d55c068ba0255c54d,"{'question_concept': 'choker', 'choices': [{'label': 'A', 'text': 'jewelry store'}, {'label': 'B', 'text': 'neck'}, {'label': 'C', 'text': 'jewlery box'}, {'label': 'D', 'text': 'jewelry box'}, {'label': 'E', 'text': 'boutique'}], 'stem': 'To locate a choker not located in a jewelry box or boutique where would you go?'}",jewelry store,To locate a choker not located in a jewelry box or boutique where would you go? A.jewelry store B.neck C.jewlery box D.jewelry box E.boutique,choker
3,D,02e821a3e53cb320790950aab4489e85,"{'question_concept': 'highway', 'choices': [{'label': 'A', 'text': 'united states'}, {'label': 'B', 'text': 'mexico'}, {'label': 'C', 'text': 'countryside'}, {'label': 'D', 'text': 'atlas'}, {'label': 'E', 'text': 'oceans'}], 'stem': 'Google Maps and other highway and street GPS services have replaced what?'}",atlas,Google Maps and other highway and street GPS services have replaced what? A.united states B.mexico C.countryside D.atlas E.oceans,highway
4,C,23505889b94e880c3e89cff4ba119860,"{'question_concept': 'fox', 'choices': [{'label': 'A', 'text': 'pretty flowers.'}, {'label': 'B', 'text': 'hen house'}, {'label': 'C', 'text': 'natural habitat'}, {'label': 'D', 'text': 'storybook'}, {'label': 'E', 'text': 'dense forest'}], 'stem': 'The fox walked from the city into the forest, what was it looking for?'}",natural habitat,"The fox walked from the city into the forest, what was it looking for? A.pretty flowers. B.hen house C.natural habitat D.storybook E.dense forest",fox


In [ ]:
test['ans_text']=test[['question','answerKey']].apply(fun_map_answerkey,axis=1)
test[['question_extracted','question_concept']]=test['question'].apply(lambda x : fun_unlist(x))

In [ ]:
train_cols=train[['question_extracted','ans_text']]
test_cols=test[['question_extracted','ans_text']]

In [ ]:
question = Field(tokenize='spacy', 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)
answer = Field(tokenize='spacy', 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)

In [ ]:
fields = [('questions', question),('answers',answer)]

In [ ]:
example = [Example.fromlist([train_cols.question_extracted[i],train_cols.ans_text[i]], fields) for i in range(train_cols.shape[0])]

In [ ]:
questionDataset = Dataset(example, fields)

In [ ]:
train=questionDataset
print(len(train))

9741


In [ ]:
print(vars(train.examples[0]))

{'questions': ['the', 'sanctions', 'against', 'the', 'school', 'were', 'a', 'punishing', 'blow', ',', 'and', 'they', 'seemed', 'to', 'what', 'the', 'efforts', 'the', 'school', 'had', 'made', 'to', 'change', '?', 'a.ignore', 'b.enforce', 'c.authoritarian', 'd.yell', 'at', 'e.avoid'], 'answers': ['ignore']}


In [ ]:
example1 = [Example.fromlist([test_cols.question_extracted[i],test_cols.ans_text[i]], fields) for i in range(test_cols.shape[0])]
valid = Dataset(example1, fields)

In [ ]:
MAX_VOCAB_SIZE = 25_000

question.build_vocab(train,max_size = MAX_VOCAB_SIZE,
                     vectors = "glove.6B.100d",unk_init = torch.Tensor.normal_)
answer.build_vocab(train,max_size = MAX_VOCAB_SIZE,
                     vectors = "glove.6B.100d",unk_init = torch.Tensor.normal_)

100%|█████████▉| 399811/400000 [00:19<00:00, 21349.03it/s]

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
BATCH_SIZE = 128

train_iterator, valid_iterator = BucketIterator.splits(
    (train, valid), 
    batch_size = BATCH_SIZE, 
    device = device,sort=False)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded)
                
        #outputs = [src len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        #initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        #outputs = [src len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        
        return outputs, hidden

In [ ]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)
        
    def forward(self, hidden, encoder_outputs):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat decoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src len, dec hid dim]
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        
        #energy = [batch size, src len, dec hid dim]

        attention = self.v(energy).squeeze(2)
        
        #attention= [batch size, src len]
        
        return F.softmax(attention, dim=1)

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs)
                
        #a = [batch size, src len]
        
        a = a.unsqueeze(1)
        
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden.squeeze(0)

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [ ]:
INPUT_DIM = len(question.vocab)
OUTPUT_DIM = len(answer.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(24877, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(3823, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=3823, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 20,635,119 trainable parameters


In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
TRG_PAD_IDX = answer.vocab.stoi[answer.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.questions
        trg = batch.answers
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.questions
            trg = batch.answers

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 7m 14s
	Train Loss: 5.484 | Train PPL: 240.801
	 Val. Loss: 4.993 |  Val. PPL: 147.363
Epoch: 02 | Time: 7m 12s
	Train Loss: 4.716 | Train PPL: 111.681
	 Val. Loss: 4.965 |  Val. PPL: 143.285
Epoch: 03 | Time: 7m 12s
	Train Loss: 4.522 | Train PPL:  92.043
	 Val. Loss: 5.006 |  Val. PPL: 149.347
Epoch: 04 | Time: 7m 13s
	Train Loss: 4.283 | Train PPL:  72.458
	 Val. Loss: 4.883 |  Val. PPL: 132.052
Epoch: 05 | Time: 7m 9s
	Train Loss: 4.034 | Train PPL:  56.469
	 Val. Loss: 4.885 |  Val. PPL: 132.331
Epoch: 06 | Time: 7m 12s
	Train Loss: 3.825 | Train PPL:  45.846
	 Val. Loss: 5.029 |  Val. PPL: 152.779
Epoch: 07 | Time: 7m 9s
	Train Loss: 3.591 | Train PPL:  36.277
	 Val. Loss: 5.250 |  Val. PPL: 190.482
Epoch: 08 | Time: 7m 8s
	Train Loss: 3.326 | Train PPL:  27.815
	 Val. Loss: 5.433 |  Val. PPL: 228.915
Epoch: 09 | Time: 7m 6s
	Train Loss: 3.041 | Train PPL:  20.935
	 Val. Loss: 5.696 |  Val. PPL: 297.727
Epoch: 10 | Time: 7m 1s
	Train Loss: 2.710 | Train PPL:  15